# Final Coursera Project - Melbourne Restaurant Analysis

### The following notebook contains my final project and report.

#### Introduction/Business Problem

I am thinking of opeining an Italian restaurant in Melbourne but am unsure where to open it. It would be best to be outside of suburbs with a lot of existing Italian restaurants but also not too far away that people would not think to look for an Italian restaurant in that location. My business problem is to find a suburb adjacent to suburbs with a strong Italian restaurant presence.

#### Data Requirements

This report will analyse the ditribution of restaurant types in Melbourne suburbs using foursquare data (venues/explore). I will find a list of Melbourne suburbs online and scrape the data. I will then use geopy.geocoders to find the latitude and longitude. I will find the number of Italian restaurants by suburb, and also the most common restaurant type by suburb and visualize the results using folium maps.

I have not completed the week 5 project yet so please disregard the code below for now. :-)

Firstly, import the required libraries

In [209]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Reading in Melbourne surbub data and finding lat long with geocode.

In [210]:
#Defining Url path
url = "https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs"

In [211]:
#Use pandas to read html
url_df = pd.read_html(url, header=0)

In [212]:
#Pull the table of suburbs from the html
df = url_df[0]
df.head()

Suburb  Postcode                              Local government area  \
0   Bellfield      3081                                    City of Banyule   
1  Briar Hill      3088                                    City of Banyule   
2    Bundoora      3083  City of Banyule; City of Darebin; City of Whit...   
3   Eaglemont      3084                                    City of Banyule   
4      Eltham      3095                City of Banyule; Shire of Nillumbik   

   Location[citation needed]  Distance[3][citation needed]  \
0                        nan                           nan   
1                        nan                           nan   
2                        nan                           nan   
3                        nan                           nan   
4                        nan                           nan   

  Area[citation needed] Population[citation needed]  \
0               0.9 km2                    1,793[4]   
1                   NaN                    3,152[4]   
2                15 km2                       28653   
3               1.9 km2                        3873   
4                   NaN                         NaN   

   Population density[citation needed]  Date established[citation needed]  
0                                  nan                                nan  
1                                  nan                                nan  
2                                  nan                                nan  
3                                  nan                                nan  
4                                  nan                                nan

In [140]:
#Format the address for geocode search
df['Address'] = df['Suburb'] + ', VIC ' + df['Postcode'].apply(str) + ", Australia"

In [141]:
#Creat a geolocator
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [142]:
#Create function to pull lat and long from location
def geoConvLat(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.latitude
    else: return 0
def geoConvLong(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.longitude
    else: return 0

In [61]:
#Find lat
#lats = []
#for i in df['Address']:
#    lats.append(geoConvLat(i))

In [62]:
#Find long
#longs = []
#for i in df['Address']:
#    longs.append(geoConvLong(i))

In [63]:
#Create lat and long columns in the df
#lats = pd.Series(lats)
#longs = pd.Series(longs)
#df['Latitude'] = lats
#df['Longitude'] = longs

In [64]:
#save the df (lat and long took a long time)
#df.to_csv('data/suburbs_and_coords.csv')

# Load Suburbs with Coordinates

In [213]:
#Read in suburbs and coords. Checkpoint
df = pd.read_csv('data/suburbs_and_coords.csv')
df.shape

(549, 13)

In [214]:
#drop irrelevant fields
drop_columns = ['Population[citation needed]', 'Area[citation needed]', 'Population density[citation needed]', 
                'Unnamed: 0', 'Location[citation needed]', 'Distance[3][citation needed]', 'Date established[citation needed]',
               'Address']
df = df.drop(drop_columns, axis=1)

In [215]:
#check the df
df.head()

Suburb  Postcode                              Local government area  \
0   Bellfield      3081                                    City of Banyule   
1  Briar Hill      3088                                    City of Banyule   
2    Bundoora      3083  City of Banyule; City of Darebin; City of Whit...   
3   Eaglemont      3084                                    City of Banyule   
4      Eltham      3095                City of Banyule; Shire of Nillumbik   

   Latitude  Longitude  
0    -37.75     145.04  
1    -37.71     145.12  
2    -37.70     145.07  
3    -37.77     145.06  
4    -37.71     145.15

In [216]:
#Plot them on a folium map
import folium

# create map using latitude and longitude values
map_melbourne = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# add markers to map
for lat, lng, suburb, lga in zip(df['Latitude'], df['Longitude'], df['Suburb'], df['Local government area']):
    label = '{}, {}'.format(suburb, lga)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

## Foursquare Section

In [12]:
#Define my details for a Foursquare query

CLIENT_ID = 'LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML' # your Foursquare ID
CLIENT_SECRET = 'AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM' # your Foursquare Secret
VERSION = '20190701' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML
CLIENT_SECRET:AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM


In [13]:
#Define the details of the query itself

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
lat, long = (-37.9136, 144.9631)
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML&client_secret=AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM&v=20190701&ll=-37.9136,144.9631&radius=1000&limit=100'

In [14]:
#Using function from Coursera labs to get nearby venues for each suburbs

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
#Getting nearby venues for each suburb

melbourne_venues = getNearbyVenues(names=df['Suburb'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [16]:
melbourne_venues.groupby('Suburb')['Suburb'].count()
print("Done")

Done


In [17]:
print('There are {} uniques categories.'.format(len(melbourne_venues['Venue Category'].unique())))

There are 215 uniques categories.


In [18]:
# one hot encoding
onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Suburb'] = melbourne_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

Suburb  Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  \
0  Bellfield               0                  0             0         0   
1  Bellfield               0                  0             0         0   
2  Bellfield               0                  0             0         0   
3  Bellfield               0                  0             0         0   
4  Bellfield               0                  0             0         0   

   Arcade  Argentinian Restaurant  Art Gallery  Asian Restaurant  \
0       0                       0            0                 0   
1       0                       0            0                 0   
2       0                       0            0                 0   
3       0                       0            0                 0   
4       0                       0            0                 0   

   Athletics & Sports        ...          Tunnel  Turkish Restaurant  \
0                   0        ...               0                   0   
1                   0        ...               0                   0   
2                   0        ...               0                   0   
3                   0        ...               0                   0   
4                   0        ...               0                   0   

   Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
0                              0            0                      0   
1                              0            0                      0   
2                              0            0                      0   
3                              0            0                      0   
4                              0            0                      0   

   Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
0           0         0          0              0                  0  
1           0         0          0              0                  0  
2           0         0          0              0                  0  
3           0         0          0              0                  0  
4           0         0          0              0                  0  

[5 rows x 216 columns]

In [19]:
onehot.shape

(8037, 216)

In [20]:
grouped = onehot.groupby('Suburb').sum().reset_index()
grouped.head()

Suburb  Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  \
0    Abbotsford               0                  0             0         0   
1    Aberfeldie               0                  1             0         0   
2       Aintree               0                  0             0         0   
3  Airport West               0                  0             0         0   
4     Albanvale               0                  0             0         0   

   Arcade  Argentinian Restaurant  Art Gallery  Asian Restaurant  \
0       0                       0            0                 1   
1       0                       0            0                 0   
2       0                       0            0                 0   
3       0                       0            0                 0   
4       0                       0            0                 0   

   Athletics & Sports        ...          Tunnel  Turkish Restaurant  \
0                   0        ...               0                   1   
1                   0        ...               0                   1   
2                   0        ...               0                   0   
3                   0        ...               0                   0   
4                   0        ...               0                   1   

   Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
0                              1            0                      1   
1                              0            0                      0   
2                              0            0                      0   
3                              0            0                      0   
4                              0            0                      0   

   Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
0           0         0          0              0                  1  
1           0         0          0              0                  0  
2           0         0          0              0                  0  
3           0         0          0              0                  0  
4           0         0          1              0                  0  

[5 rows x 216 columns]

In [21]:
grouped.shape

(307, 216)

In [22]:
grouped.head()

Suburb  Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  \
0    Abbotsford               0                  0             0         0   
1    Aberfeldie               0                  1             0         0   
2       Aintree               0                  0             0         0   
3  Airport West               0                  0             0         0   
4     Albanvale               0                  0             0         0   

   Arcade  Argentinian Restaurant  Art Gallery  Asian Restaurant  \
0       0                       0            0                 1   
1       0                       0            0                 0   
2       0                       0            0                 0   
3       0                       0            0                 0   
4       0                       0            0                 0   

   Athletics & Sports        ...          Tunnel  Turkish Restaurant  \
0                   0        ...               0                   1   
1                   0        ...               0                   1   
2                   0        ...               0                   0   
3                   0        ...               0                   0   
4                   0        ...               0                   1   

   Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
0                              1            0                      1   
1                              0            0                      0   
2                              0            0                      0   
3                              0            0                      0   
4                              0            0                      0   

   Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
0           0         0          0              0                  1  
1           0         0          0              0                  0  
2           0         0          0              0                  0  
3           0         0          0              0                  0  
4           0         0          1              0                  0  

[5 rows x 216 columns]

In [39]:
merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(grouped.set_index('Suburb'), on='Suburb')

merged.tail()

Suburb  Postcode  Local government area   Latitude   Longitude  \
544   Woori Yallock      3139  Shire of Yarra Ranges -37.779333  145.530126   
545      Yarra Glen      3775  Shire of Yarra Ranges -37.657348  145.374396   
546  Yarra Junction      3797  Shire of Yarra Ranges -37.782169  145.615026   
547       Yellingbo      3139  Shire of Yarra Ranges -37.813513  145.508205   
548          Yering      3770  Shire of Yarra Ranges -37.688713  145.374657   

     Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  Arcade  \
544             0.0                0.0           0.0       0.0     0.0   
545             0.0                0.0           0.0       0.0     0.0   
546             0.0                0.0           0.0       0.0     0.0   
547             0.0                0.0           0.0       0.0     0.0   
548             0.0                0.0           0.0       0.0     0.0   

           ...          Tunnel  Turkish Restaurant  \
544        ...             0.0                 0.0   
545        ...             0.0                 0.0   
546        ...             0.0                 0.0   
547        ...             0.0                 0.0   
548        ...             0.0                 0.0   

     Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
544                            0.0          1.0                    1.0   
545                            0.0          0.0                    0.0   
546                            0.0          0.0                    0.0   
547                            1.0          0.0                    0.0   
548                            0.0          0.0                    0.0   

     Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
544         0.0       0.0        0.0            0.0                0.0  
545         0.0       0.0        0.0            0.0                0.0  
546         0.0       0.0        0.0            0.0                0.0  
547         1.0       1.0        0.0            0.0                0.0  
548         0.0       0.0        0.0            0.0                0.0  

[5 rows x 220 columns]

In [40]:
merged = merged.fillna(0)

In [41]:
merged.to_csv('data/mergedData.csv', index=None)

# Load Suburbs Merged with Venues

In [217]:
merged= pd.read_csv('data/mergedData.csv')
keepSuburbs = pd.read_csv('data/keepSuburbs.csv')
keepVenue = pd.read_csv('data/venues.csv')

In [218]:
keepList = []
for i in keepSuburbs['Suburb']:
    keepList.append(i)

keepList[:2]

['Beaumaris', 'Black Rock']

In [219]:
merged.head()

Suburb  Postcode                              Local government area  \
0   Bellfield      3081                                    City of Banyule   
1  Briar Hill      3088                                    City of Banyule   
2    Bundoora      3083  City of Banyule; City of Darebin; City of Whit...   
3   Eaglemont      3084                                    City of Banyule   
4      Eltham      3095                City of Banyule; Shire of Nillumbik   

   Latitude  Longitude  Adult Boutique  Afghan Restaurant  Antique Shop  \
0    -37.75     145.04            0.00               0.00          0.00   
1    -37.71     145.12            0.00               0.00          0.00   
2    -37.70     145.07            0.00               0.00          0.00   
3    -37.77     145.06            0.00               1.00          0.00   
4    -37.71     145.15            0.00               0.00          0.00   

   Aquarium  Arcade  Argentinian Restaurant  Art Gallery  Asian Restaurant  \
0      0.00    0.00                    0.00         0.00              0.00   
1      0.00    0.00                    0.00         0.00              0.00   
2      0.00    0.00                    0.00         0.00              0.00   
3      0.00    0.00                    0.00         0.00              0.00   
4      0.00    0.00                    0.00         0.00              0.00   

   Athletics & Sports  Australian Restaurant  BBQ Joint  Bagel Shop  Bakery  \
0                0.00                   0.00       0.00        0.00    0.00   
1                0.00                   0.00       0.00        0.00    0.00   
2                0.00                   0.00       0.00        0.00    0.00   
3                0.00                   0.00       0.00        0.00    1.00   
4                0.00                   0.00       0.00        0.00    0.00   

   Bar  Basketball Court  Beach  Beer Garden  Bike Rental / Bike Share  \
0 1.00              0.00   0.00         1.00                      0.00   
1 0.00              0.00   0.00         0.00                      0.00   
2 0.00              0.00   0.00         0.00                      0.00   
3 4.00              0.00   0.00         1.00                      0.00   
4 0.00              0.00   0.00         0.00                      0.00   

   Boat or Ferry  Bookstore  Boutique  Bowling Alley  Brazilian Restaurant  \
0           0.00       0.00      0.00           0.00                  0.00   
1           0.00       0.00      0.00           0.00                  0.00   
2           0.00       0.00      0.00           0.00                  0.00   
3           0.00       1.00      0.00           0.00                  0.00   
4           0.00       0.00      0.00           0.00                  0.00   

   Breakfast Spot  Brewery  Bubble Tea Shop  Burger Joint  Burrito Place  \
0            0.00     0.00             0.00          0.00           0.00   
1            0.00     0.00             0.00          0.00           0.00   
2            0.00     0.00             0.00          0.00           0.00   
3            1.00     0.00             0.00          0.00           0.00   
4            0.00     0.00             0.00          0.00           0.00   

   Bus Station  Business Service  Café  Camera Store  Candy Store  \
0         0.00              0.00  0.00          0.00         0.00   
1         0.00              0.00  0.00          0.00         0.00   
2         0.00              0.00  0.00          0.00         0.00   
3         0.00              0.00 10.00          0.00         0.00   
4         0.00              0.00  0.00          0.00         0.00   

   Cantonese Restaurant  Casino  Cemetery  Cheese Shop  Chinese Restaurant  \
0                  0.00    0.00      0.00         0.00                0.00   
1                  0.00    0.00      0.00         0.00                0.00   
2                  0.00    0.00      0.00         0.00                0.00   
3                  0.00    0.00      0.00    

In [220]:
keepVenues = ['Suburb', 'Postcode', 'Local government area', 'Latitude', 'Longitude']
for i in keepVenue['Venue_Type']:
    keepVenues.append(i)

keepVenues[:2]

['Suburb', 'Postcode']

In [221]:
merged = merged[keepVenues]
merged = merged[merged['Suburb'].isin(keepList)]
merged = merged[merged['Suburb'].isin(keepList)]

In [222]:
merged['Italian Restaurant'].unique()

array([ 0.,  1.,  3.,  2., 14.,  5.,  4., 10.])

In [223]:
merged.shape

(78, 49)

In [224]:
house_prices = pd.read_excel('data/melbourneSalesPrices.xlsx', index=None)

In [225]:
house_prices.shape

(48427, 2)

In [226]:
houses_grouped = house_prices.groupby('Suburb').mean().reset_index()
houses_grouped.sort_values(by='Price')[:5]

Suburb     Price
108           Darley 380000.00
204        Kurunjang 381785.71
224     Melton South 396267.50
90          Cockatoo 397000.00
240  Mount Dandenong 405000.00

In [227]:
joined_df = merged.join(houses_grouped.set_index('Suburb'), on='Suburb', how='inner')
joined_df.head()

Suburb  Postcode              Local government area  Latitude  \
21      Beaumaris      3193                    City of Bayside    -37.98   
22     Black Rock      3193                    City of Bayside    -37.97   
23       Brighton      3186                    City of Bayside    -37.91   
24  Brighton East      3187                    City of Bayside    -37.92   
25     Cheltenham      3192  City of Bayside; City of Kingston    -37.96   

    Longitude  Middle Eastern Restaurant  Mediterranean Restaurant  \
21     145.04                       0.00                      0.00   
22     145.02                       0.00                      0.00   
23     145.00                       0.00                      0.00   
24     145.02                       0.00                      0.00   
25     145.06                       0.00                      0.00   

    Indian Restaurant  Japanese Restaurant  Portuguese Restaurant  \
21               0.00                 0.00                   0.00   
22               0.00                 0.00                   0.00   
23               0.00                 0.00                   0.00   
24               0.00                 0.00                   0.00   
25               0.00                 0.00                   0.00   

    Fast Food Restaurant  Lebanese Restaurant  Mexican Restaurant  \
21                  0.00                 0.00                0.00   
22                  0.00                 0.00                0.00   
23                  0.00                 0.00                0.00   
24                  0.00                 0.00                0.00   
25                  0.00                 0.00                0.00   

    Falafel Restaurant  Vietnamese Restaurant  Thai Restaurant  \
21                0.00                   0.00             0.00   
22                0.00                   0.00             0.00   
23                0.00                   0.00             0.00   
24                0.00                   0.00             0.00   
25                0.00                   0.00             0.00   

    Turkish Restaurant  Afghan Restaurant  Eastern European Restaurant  \
21                0.00               0.00                         0.00   
22                0.00               0.00                         0.00   
23                0.00               0.00                         0.00   
24                0.00               0.00                         0.00   
25                0.00               0.00                         0.00   

    Dumpling Restaurant  Sushi Restaurant  Kebab Restaurant  \
21                 0.00              0.00              0.00   
22                 0.00              0.00              0.00   
23                 0.00              0.00              0.00   
24                 0.00              0.00              0.00   
25                 0.00              0.00              0.00   

    Italian Restaurant  Indonesian Restaurant  Korean Restaurant  \
21                0.00                   0.00               0.00   
22                0.00                   0.00               0.00   
23                0.00                   0.00               0.00   
24                0.00                   0.00               0.00   
25                0.00                   0.00               0.00   

    Ramen Restaurant  Dim Sum Restaurant  Chinese Restaurant  \
21              0.00                0.00                0.00   
22              0.00                0.00                0.00   
23              0.00                0.00                0.00   
24              0.00                0.00                0.00   
25              0.00                0.00                0.00   

    Malay Restaurant  Brazilian Restaurant  Vegetarian / Vegan Restaurant  \
21              0.00                  0.00                           0.00   
22              0.00                  0.00                           0.00   
23              0.00                  0.00               

In [228]:
# set number of clusters
kclusters = 5

merged_clustering = joined_df.drop(['Suburb', 'Postcode', 'Local government area', 'Latitude', 'Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42, n_init=100).fit(merged_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 0, 3, 1, 1, 1, 3, 1, 1, 1, 3, 3, 0, 3, 3, 0, 0, 3, 2, 2, 3,
       3, 2, 3, 2, 1, 4, 1, 1, 2, 4, 2, 4, 2, 1, 1, 2, 2, 4, 2, 4, 4, 4,
       2, 2, 4, 1, 1, 0, 4, 1, 0, 4, 4, 2, 3, 3, 0, 3, 1, 3, 1, 1, 1, 2,
       4, 1, 2, 2, 3, 1])

In [229]:
joined_df.insert(column ='Cluster', value = kmeans.labels_, loc = 1)
joined_df.head()

Suburb  Cluster  Postcode              Local government area  \
21      Beaumaris        3      3193                    City of Bayside   
22     Black Rock        3      3193                    City of Bayside   
23       Brighton        0      3186                    City of Bayside   
24  Brighton East        3      3187                    City of Bayside   
25     Cheltenham        1      3192  City of Bayside; City of Kingston   

    Latitude  Longitude  Middle Eastern Restaurant  Mediterranean Restaurant  \
21    -37.98     145.04                       0.00                      0.00   
22    -37.97     145.02                       0.00                      0.00   
23    -37.91     145.00                       0.00                      0.00   
24    -37.92     145.02                       0.00                      0.00   
25    -37.96     145.06                       0.00                      0.00   

    Indian Restaurant  Japanese Restaurant  Portuguese Restaurant  \
21               0.00                 0.00                   0.00   
22               0.00                 0.00                   0.00   
23               0.00                 0.00                   0.00   
24               0.00                 0.00                   0.00   
25               0.00                 0.00                   0.00   

    Fast Food Restaurant  Lebanese Restaurant  Mexican Restaurant  \
21                  0.00                 0.00                0.00   
22                  0.00                 0.00                0.00   
23                  0.00                 0.00                0.00   
24                  0.00                 0.00                0.00   
25                  0.00                 0.00                0.00   

    Falafel Restaurant  Vietnamese Restaurant  Thai Restaurant  \
21                0.00                   0.00             0.00   
22                0.00                   0.00             0.00   
23                0.00                   0.00             0.00   
24                0.00                   0.00             0.00   
25                0.00                   0.00             0.00   

    Turkish Restaurant  Afghan Restaurant  Eastern European Restaurant  \
21                0.00               0.00                         0.00   
22                0.00               0.00                         0.00   
23                0.00               0.00                         0.00   
24                0.00               0.00                         0.00   
25                0.00               0.00                         0.00   

    Dumpling Restaurant  Sushi Restaurant  Kebab Restaurant  \
21                 0.00              0.00              0.00   
22                 0.00              0.00              0.00   
23                 0.00              0.00              0.00   
24                 0.00              0.00              0.00   
25                 0.00              0.00              0.00   

    Italian Restaurant  Indonesian Restaurant  Korean Restaurant  \
21                0.00                   0.00               0.00   
22                0.00                   0.00               0.00   
23                0.00                   0.00               0.00   
24                0.00                   0.00               0.00   
25                0.00                   0.00               0.00   

    Ramen Restaurant  Dim Sum Restaurant  Chinese Restaurant  \
21              0.00                0.00                0.00   
22              0.00                0.00                0.00   
23              0.00                0.00                0.00   
24              0.00                0.00                0.00   
25              0.00                0.00                0.00   

    Malay Restaurant  Brazilian Restaurant  Vegetarian / Vegan Restaurant  \
21              0.00                  0.00                           0.00   
22              0.00                  0.00                           0.00   
23 

In [230]:
# create map
map_clusters = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
import math
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, n, p in zip(joined_df['Latitude'], joined_df['Longitude'], joined_df['Suburb'], 
                                     joined_df['Cluster'], joined_df['Italian Restaurant'], joined_df['Price']):
    label = folium.Popup(str(poi) + ", Cluster: " + str(cluster) + ", Price: " + str(p//1) + 
                         ", Italian Restuarants: " + str(int(n)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=1
    ).add_to(map_clusters)
    
       
map_clusters

In [231]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [232]:
joined_df.loc[joined_df['Cluster'] == 0].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count     7.00      7.00      7.00       7.00                       7.00   
mean      0.00   3153.43    -37.84     145.03                       0.00   
std       0.00     45.67      0.03       0.05                       0.00   
min       0.00   3103.00    -37.91     144.96                       0.00   
25%       0.00   3114.50    -37.85     144.98                       0.00   
50%       0.00   3144.00    -37.85     145.04                       0.00   
75%       0.00   3196.00    -37.82     145.07                       0.00   
max       0.00   3206.00    -37.81     145.08                       0.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                      7.00               7.00                 7.00   
mean                       0.00               0.14                 0.86   
std                        0.00               0.38                 1.07   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 2.00   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                   7.00                  7.00                 7.00   
mean                    0.29                  0.57                 0.00   
std                     0.76                  0.98                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  1.00                 0.00   
max                     2.00                  2.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count                7.00                7.00                   7.00   
mean                 0.71                0.00                   0.00   
std                  1.25                0.00                   0.00   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  1.00                0.00                   0.00   
max                  3.00                0.00                   0.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count             7.00                7.00               7.00   
mean              0.29                0.00               0.00   
std               0.49                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               0.50                0.00               0.00   
max               1.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                         7.00                 7.00              7.00   
mean                          0.00                 0.29              0.57   
std                           0.00                 0.76              0.79   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              1.00   
max                           0.00                 2.00              2.00   

       Kebab Restaurant  Italian Restaura

In [233]:
joined_df.loc[joined_df['Cluster'] == 1].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    20.00     20.00     20.00      20.00                      20.00   
mean      1.00   3154.35    -37.88     145.02                       0.10   
std       0.00     49.22      0.05       0.04                       0.31   
min       1.00   3003.00    -37.96     144.92                       0.00   
25%       1.00   3136.00    -37.91     145.00                       0.00   
50%       1.00   3173.00    -37.88     145.01                       0.00   
75%       1.00   3185.75    -37.84     145.04                       0.00   
max       1.00   3204.00    -37.80     145.07                       1.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     20.00              20.00                20.00   
mean                       0.00               0.10                 0.40   
std                        0.00               0.31                 0.99   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 0.00   
max                        0.00               1.00                 3.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  20.00                 20.00                20.00   
mean                    0.10                  0.10                 0.00   
std                     0.31                  0.45                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     1.00                  2.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               20.00               20.00                  20.00   
mean                 0.25                0.00                   0.10   
std                  0.72                0.00                   0.31   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  0.00                0.00                   0.00   
max                  3.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            20.00               20.00              20.00   
mean              0.40                0.05               0.00   
std               0.94                0.22               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               0.25                0.00               0.00   
max               4.00                1.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        20.00                20.00             20.00   
mean                          0.00                 0.05              0.15   
std                           0.00                 0.22              0.49   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              0.00   
max                           0.00                 1.00              2.00   

       Kebab Restaurant  Italian Restaura

In [234]:
joined_df.loc[joined_df['Cluster'] == 2].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    16.00     16.00     16.00      16.00                      16.00   
mean      2.00   3118.81    -37.83     145.00                       0.06   
std       0.00     68.68      0.05       0.05                       0.25   
min       2.00   3002.00    -37.92     144.92                       0.00   
25%       2.00   3062.25    -37.85     144.97                       0.00   
50%       2.00   3122.50    -37.82     144.99                       0.00   
75%       2.00   3189.75    -37.80     145.03                       0.00   
max       2.00   3207.00    -37.78     145.11                       1.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     16.00              16.00                16.00   
mean                       0.00               0.06                 0.50   
std                        0.00               0.25                 0.89   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 0.50   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  16.00                 16.00                16.00   
mean                    0.06                  0.06                 0.00   
std                     0.25                  0.25                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     1.00                  1.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               16.00               16.00                  16.00   
mean                 0.50                0.00                   0.12   
std                  0.82                0.00                   0.34   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  1.00                0.00                   0.00   
max                  3.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            16.00               16.00              16.00   
mean              0.69                0.00               0.00   
std               0.79                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.50                0.00               0.00   
75%               1.00                0.00               0.00   
max               2.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        16.00                16.00             16.00   
mean                          0.00                 0.19              0.56   
std                           0.00                 0.54              0.63   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.50   
75%                           0.00                 0.00              1.00   
max                           0.00                 2.00              2.00   

       Kebab Restaurant  Italian Restaura

In [235]:
joined_df.loc[joined_df['Cluster'] == 3].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    17.00     17.00     17.00      17.00                      17.00   
mean      3.00   3143.00    -37.87     145.04                       0.00   
std       0.00     39.37      0.06       0.03                       0.00   
min       3.00   3054.00    -37.98     144.97                       0.00   
25%       3.00   3124.00    -37.92     145.02                       0.00   
50%       3.00   3144.00    -37.86     145.04                       0.00   
75%       3.00   3187.00    -37.83     145.07                       0.00   
max       3.00   3193.00    -37.78     145.10                       0.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     17.00              17.00                17.00   
mean                       0.00               0.06                 0.12   
std                        0.00               0.24                 0.49   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 0.00   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  17.00                 17.00                17.00   
mean                    0.00                  0.06                 0.00   
std                     0.00                  0.24                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     0.00                  1.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               17.00               17.00                  17.00   
mean                 0.00                0.00                   0.06   
std                  0.00                0.00                   0.24   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  0.00                0.00                   0.00   
max                  0.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            17.00               17.00              17.00   
mean              0.47                0.00               0.00   
std               1.01                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               1.00                0.00               0.00   
max               4.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        17.00                17.00             17.00   
mean                          0.00                 0.00              0.18   
std                           0.00                 0.00              0.53   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              0.00   
max                           0.00                 0.00              2.00   

       Kebab Restaurant  Italian Restaura

In [236]:
joined_df.loc[joined_df['Cluster'] == 4].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    12.00     12.00     12.00      12.00                      12.00   
mean      4.00   3102.58    -37.84     144.98                       0.08   
std       0.00     75.63      0.04       0.04                       0.29   
min       4.00   3006.00    -37.89     144.92                       0.00   
25%       4.00   3031.00    -37.88     144.94                       0.00   
50%       4.00   3114.00    -37.84     144.98                       0.00   
75%       4.00   3167.75    -37.80     145.00                       0.00   
max       4.00   3185.00    -37.79     145.06                       1.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     12.00              12.00                12.00   
mean                       0.00               0.25                 0.50   
std                        0.00               0.45                 0.80   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.25                 1.00   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  12.00                 12.00                12.00   
mean                    0.08                  0.00                 0.00   
std                     0.29                  0.00                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     1.00                  0.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               12.00               12.00                  12.00   
mean                 0.17                0.00                   0.17   
std                  0.39                0.00                   0.39   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  0.00                0.00                   0.00   
max                  1.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            12.00               12.00              12.00   
mean              0.42                0.00               0.00   
std               0.79                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               0.25                0.00               0.00   
max               2.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        12.00                12.00             12.00   
mean                          0.00                 0.08              0.42   
std                           0.00                 0.29              0.51   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              1.00   
max                           0.00                 1.00              1.00   

       Kebab Restaurant  Italian Restaura

In [237]:
cluster_ave_price = pd.DataFrame([joined_df.loc[joined_df['Cluster'] == 0]['Price'].mean(), joined_df.loc[joined_df['Cluster'] == 1]['Price'].mean(),
joined_df.loc[joined_df['Cluster'] == 2]['Price'].mean(), joined_df.loc[joined_df['Cluster'] == 3]['Price'].mean(),
joined_df.loc[joined_df['Cluster'] == 4]['Price'].mean()],columns=['Average Price'])
cluster_ave_price

Average Price
0     2104943.87
1     1067327.42
2     1332251.53
3     1647958.21
4      815137.76

In [238]:
cluster_ave_italian = pd.DataFrame([joined_df.loc[joined_df['Cluster'] == 0]['Italian Restaurant'].mean(), 
joined_df.loc[joined_df['Cluster'] == 1]['Italian Restaurant'].mean(),
joined_df.loc[joined_df['Cluster'] == 2]['Italian Restaurant'].mean(), 
joined_df.loc[joined_df['Cluster'] == 3]['Italian Restaurant'].mean(),
joined_df.loc[joined_df['Cluster'] == 4]['Italian Restaurant'].mean()], columns=['Ave Italian Restaurants'])
cluster_ave_italian

Ave Italian Restaurants
0                     0.86
1                     0.80
2                     1.12
3                     0.82
4                     1.08

In [239]:
joined_df.loc[joined_df['Cluster'] == 1][['Suburb', 'Italian Restaurant', 'Price']]

Suburb  Italian Restaurant      Price
25        Cheltenham                0.00  969127.39
26        Gardenvale                0.00 1095312.50
169       Gardenvale                0.00 1095312.50
28      Hampton East                0.00 1096513.04
29           Highett                0.00 1083245.06
30         Moorabbin                0.00 1009232.99
162   Bentleigh East                0.00 1132489.32
164   Caulfield East                0.00 1035750.00
165  Caulfield North                0.00  958154.49
172      Murrumbeena                0.00 1061576.90
173           Ormond                0.00 1096426.63
295      South Yarra                0.00 1078120.07
296   West Melbourne                1.00 1041723.08
429           Elwood                0.00 1011153.37
439          Prahran                0.00 1149590.55
441          Windsor                0.00 1051843.75
478       Abbotsford               10.00 1040200.70
479          Burnley                0.00 1191875.62
482         Cremorne                0.00 1076294.12
486         Richmond                5.00 1072606.23

In [255]:
best_suburb = joined_df.loc[joined_df['Cluster'] == 1][['Suburb', 'Cluster', 'Latitude', 'Longitude', 
                                                        'Postcode', 'Local government area', 'Italian Restaurant', 'Price']]
best_suburb.loc[best_suburb['Suburb'] == 'Burnley']

Suburb  Cluster  Latitude  Longitude  Postcode Local government area  \
479  Burnley        1    -37.83     145.02      3121         City of Yarra   

     Italian Restaurant      Price  
479                0.00 1191875.62

In [279]:
# create map using latitude and longitude values
map_burnley = folium.Map(location=(-37.83, 145.02), zoom_start=13)

label = '{}, {}'.format('Burnley', 'City of Yarra')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [-37.83, 145.0175],
    radius=40,
    popup=label,
    color='blue',
    fill=True,
    fill_opacity=0.2,
    parse_html=False).add_to(map_burnley)  

map_burnley